# Network data exploration

## Load data

Gather paths from private json file

In [ ]:
import json

with open('paths.json') as f:
    data = json.load(f)
data

In [ ]:
networkfile = data['data']['file']

Read network data from file

In [ ]:
import scapy.all

a = scapy.all.rdpcap(networkfile)
sessions = a.sessions()

print('Loaded {} sessions'.format(len(sessions)))

## Investigate pcap data

### Investigate single packet list

In [ ]:
print(next(iter(sessions.keys())), ' : ' ,next(iter(sessions.values())))
print(type(next(iter(sessions.keys()))), ' : ' ,type(next(iter(sessions.values()))))

In [ ]:
packetlist = next(iter(sessions.values()))

In [ ]:
packetlist.show()